# Example: Modeling the Premium, Payout, and Profit of Long and Short American-style Call Contracts
This example will familiarize students with calculating the premium, profit, and breakeven for [American-style](https://en.wikipedia.org/wiki/Option_style) `call` contracts. 

### Learning objectives
* __Objective 1__: is focused on computing the premium of American-style `call` contracts using the [Cox, Ross, and Rubinstein (CRR) model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model), a specific Binomial lattice model implementation.
* __Objective 2__: is focused on computing the payoff, profit, and breakeven points for American-style `call` contracts. 

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* See the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl) for additional information on functions and types used in this material. 

In [1]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-2/Project.toml`
  [4119e0bf] + VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-2/Manifest.toml`
  [14f7f29c] + AMD v0.5.3
  [79e6a3ab] + Adapt v4.0.1
  [4fba245c] + ArrayInterface v7.7.1
  [6e4b80f9] + BenchmarkTools v1.4.0
  [336ed68f] + CSV v0.10.12
  [49dc2e85] + Calculus v0.5.1
  [523fee87] + CodecBzip2 v0.8.2
  [bbf7d656] + CommonSubexpressions v0.3.0
  [187b0558] + ConstructionBase v1.5.4
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.15.1
  [31c24e10] + Distributions v0.25.107
  [fa6b7ba4] + DualNumbers v0.6.8
  [e2ba6199] + ExprTools v0.1.10
  [9aa1b823] + FastClosures v0.3.2
  [5789e2e9] + FileIO v1.16.

## Objective 1: Compute the Premium of American-style `call` contracts using a Binomial lattice
Let's consider an American-style contract with the same parameters as the previous European contract, except that early exercise is now allowed. In particular, let's model a `call` that expires in `T = 365` days with a strike price of `K = 60.0 USD/share`. The current share price of the underlying asset is `S(0) = 60.0 USD/share`. Assume a risk-free rate of return of `5%` and an implied volatility of `10%`. The `European-style` contract price is $\mathcal{P}_{c}(K,S(0))$ = `4.08 USD/share`.
* To start this calculation, we first set the parameters for this case and then build an instance of the `MyAdjacencyBasedCRREquityPriceTree` type, which we use to compute the price. 

In [2]:
Δt = (1.0/365.0);
Sₒ = 60.0;
K = 60.0;
T = 365.0*Δt;
r̄ = 0.05;
σ̄ = 0.10;

Next, let's build an instance of the `MyAdjacencyBasedCRREquityPriceTree`, which holds the problem parameters, and store this in the `treemodel` variable. We make this model using the `build(...)` function, where we pass the model type and problem information as arguments to the `build(...)` function:

In [5]:
treemodel = build(MyAdjacencyBasedCRREquityPriceTree, (μ = r̄, T = T, σ = σ̄, 
        Sₒ = Sₒ, h = 365 # h holds the number of levels in the tree
));

In [6]:
treemodel

MyAdjacencyBasedCRREquityPriceTree(0.05, 0.1, 1.0, nothing, nothing, nothing, nothing, nothing, nothing, nothing)